In [1]:
import numpy as np
import pandas as pd
import h5py
import csv
from DataGenerator import DataGenerator
from Models import Models
from LoadData import LoadData
from BaselineHelperFunctions import BaselineHelperFunctions
from RandomGridSearch import RandomGridSearch
import tensorflow as tf

Using TensorFlow backend.


In [2]:
csv_root = 'csv_folder_3_class'
full_data_csv, train_csv, val_csv, test_csv = LoadData(csv_root, isBalanced = True).getData()
data_gen = DataGenerator(csv_root, train_csv, val_csv, test_csv)
full_ds = data_gen.load_dataset(full_data_csv)
train_ds = data_gen.load_dataset(train_csv)
val_ds = data_gen.load_dataset(val_csv)
test_ds = data_gen.load_dataset(test_csv)


In [3]:
def get_trace_shape_no_cast(ds):
    num_ds = len(ds)
    with h5py.File(ds[0][0], 'r') as dp:
        trace_shape = dp.get('traces').shape
    return num_ds, trace_shape[0], trace_shape[1]

In [4]:
get_trace_shape_no_cast(val_ds)

(2878, 3, 6001)

In [5]:
def load_datasets(data_csv):
        nr_rows = 0
        with open(data_csv, 'r') as file:
            csv_reader = csv.reader(file, delimiter = ',')
            nr_rows = len(list(csv_reader))
        dataset = np.empty((nr_rows, 2), dtype=object)
        with open(data_csv, 'r') as file:
            csv_reader = csv.reader(file, delimiter = ',')
            idx = 0
            for row in csv_reader:
                dataset[idx][0] = row[0]
                dataset[idx][1] = row[1]
                idx += 1
            return dataset

In [6]:
df_2 = load_datasets(train_csv)

In [7]:
np.unique(df_2[16444:16445])

array([], dtype=object)

In [8]:
def load_dataset(data_csv):
    columns = ["path", "label"]
    df = pd.read_csv(data_csv, names = columns)
    return df.values

In [9]:
df = load_dataset(train_csv)

In [10]:
len(df)
df[16444:16445]

array([], shape=(0, 2), dtype=object)

In [11]:
model_nr = 4
    
batch_size = 128
epochs = 100
learning_rate = 0.0001
opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
activation = 'tanh'
dropout_rate = 0.2
l1_r = 0.2
l2_r = 0.001
start_neurons = 512

test = False
num_classes = 3
detrend = False
gen_args = {
    'batch_size' : batch_size,
    'shuffle' : True,
    'test' : test,
    'detrend' : False,
    'num_classes' : num_classes,
}


In [12]:
train_gen = data_gen.data_generator(train_ds, **gen_args)
val_gen = data_gen.data_generator(val_ds, **gen_args)

In [13]:
def check_labels(y, batch_size):
    if not np.sum(y) == batch_size:
        print(y)
        return True
    else:
        return False


def check_generator(gen, batch_size, nr_steps, train = True):
    i = 0
    for x_y in gen:
        if i % 10 == 0 || i == nr_steps:
            if train:
                print(f'{i}/{nr_steps}')
            else:
                print(f'{i}/{nr_steps_val}')
        if not i == nr_steps:
            y = x_y[1]
            check_labels(y, batch_size)
            i += 1
        else:
            break

nr_steps = len(train_ds) // batch_size
nr_steps_val = len(val_ds) // batch_size
check_generator(train_gen, batch_size, nr_steps)
check_generator(val_gen, batch_size, nr_steps_val, False)
    

SyntaxError: invalid syntax (<ipython-input-13-0cc0e6a7ba0b>, line 12)